In [31]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
import json
import pickle
import io
import pandas as pd
from dataclasses import asdict
from backdoors import data, patterns, poison, utils, checkpoint_dir
from backdoors.data import load_cifar10, Data
import numpy as np
import matplotlib.pyplot as plt
import einops
import jax
from jax import numpy as jnp
from flax.training import train_state
import optax
import chex
import flax
from flax import linen as nn
from typing import Tuple
import matplotlib.pyplot as plt
import sympy
from jax import random
from collections import namedtuple
from time import time

import orbax.checkpoint
import os
from pathlib import Path

from backdoors import image_utils
from backdoors import module_path, checkpoint_dir, train, paths
from backdoors.train import TrainState, Metrics, model, tx, \
    accuracy, train_step, init_train_state
from backdoors.models import CNN
from meta_transformer.data import load_pair_of_models, data_iterator
from tqdm import tqdm
rng = random.PRNGKey(0)

checkpointer = orbax.checkpoint.PyTreeCheckpointer()

BATCH_SIZE = 64
NUM_EPOCHS = 5

#train_data, _ = load_cifar10()
#cifar10_train, cifar10_test = load_cifar10()

In [32]:
# load batch
def load_batch(filename):
    with open(filename, 'rb') as f:
        batch = pickle.load(f)
    return batch

In [ ]:
def load_batches(datadir):
    """Load all batches from a directory"""
    for entry in os.scandir(datadir):
        if entry.name.startswith('checkpoints'):
            yield load_batch(entry.path)

In [ ]:
def load_models(idxs: Sequence[int], dir: str, max_workers: int = 1):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        out = executor.map(partial(load_model, dir=dir), idxs)
    models, info = [list(x) for x in zip(*out)]
    return models, info